# Purpose

### 2022-04-06
In this notebook we'll get the subreddits relevant to a country so that we can do QA (check ratings & subreddit relevance) for an FPR in one Feed.

---

TODO: Haven't included place logic (e.g., add direction to: city, state, country subreddits.)


### Updates
2022-02-23:
I'm going to automate more of the notebook process into functions:
- automate "optimal" minimum number of subs to minimize number of orphans
- move df_clean creation & column order to a function
- update order of columns for cluster dfs

2022-02-16: 
I [David] will update the QA sheet so that we have standardized columns/format. Otherwise it'll be more work for us to wait for country managers to format things and then standardize them after the fact



# Imports & notebook setup

In [2]:
%load_ext autoreload
%autoreload 2

# Register bigquery magic (only needed for laptop/local, not colab)
# %load_ext google.cloud.bigquery

In [3]:
# colab auth for BigQuery, google drive, & google sheets (gspread)
from google.colab import auth, files, drive
from google.auth import default
import sys  # need sys for mounting gdrive path

auth.authenticate_user()
print('Authenticated')

Authenticated


## Install custom library

### Append google drive path so we can install library from there

In [4]:
# Attach google drive & import my python utility functions
# if drive.mount() fails, you can also:
#   MANUALLY CLICK ON "Mount Drive"
import sys


g_drive_root = '/content/drive'

try:
    drive.mount(g_drive_root, force_remount=True)
    print('   Authenticated & mounted Google Drive')
    
except Exception as e:
    try:
        drive._mount(g_drive_root, force_remount=True)
        print('   Authenticated & mounted Google Drive')
    except Exception as e:
        print(e)
        raise Exception('You might need to manually mount google drive to colab')

l_paths_to_append = [
    f'{g_drive_root}/MyDrive/Colab Notebooks',

    # need to append the path to subclu so that colab can import things properly
    f'{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n'
]
for path_ in l_paths_to_append:
    if path_ in sys.path:
        sys.path.remove(path_)
    print(f" Appending path: {path_}")
    sys.path.append(path_)

Mounted at /content/drive
   Authenticated & mounted Google Drive
 Appending path: /content/drive/MyDrive/Colab Notebooks
 Appending path: /content/drive/MyDrive/Colab Notebooks/subreddit_clustering_i18n


### Install library

In [5]:
# install subclu & libraries needed to read parquet files from GCS & spreadsheets
#  make sure to use the [colab] `extra` because it includes colab-specific libraries
module_path = f"{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n/[colab]"

!pip install -e $"$module_path" --quiet

     |████████████████████████████████| 10.1 MB 5.2 MB/s 
     |████████████████████████████████| 14.2 MB 32.7 MB/s 
     |████████████████████████████████| 965 kB 61.6 MB/s 
     |████████████████████████████████| 144 kB 62.9 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 285 kB 56.8 MB/s 
     |████████████████████████████████| 13.2 MB 38.2 MB/s 
     |████████████████████████████████| 79.9 MB 83 kB/s 
     |████████████████████████████████| 136 kB 45.2 MB/s 
     |████████████████████████████████| 715 kB 45.8 MB/s 
     |████████████████████████████████| 74 kB 3.1 MB/s 
     |████████████████████████████████| 112 kB 43.9 MB/s 
     |████████████████████████████████| 62 kB 626 kB/s 
     |████████████████████████████████| 79 kB 5.4 MB/s 
     |████████████████████████████████| 181 kB 55.6 MB/s 
     |████████████████████████████████| 1.1 MB 48.0 MB/s 
     |████████████████████████████████| 146 kB 52.1 MB/s 
     |███████████████

## Regular Imports

In [6]:
import os
from datetime import datetime

from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib_venn import venn2_unweighted, venn3_unweighted
from tqdm import tqdm

# auth for google sheets
import gspread


creds_, _ = default()
gc = gspread.authorize(creds_)

# os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-science-prod-218515'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'

## Custom imports

In [7]:
# subclu imports
import subclu
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.models.clustering_utils import (
    create_dynamic_clusters,
    convert_distance_or_ab_to_list_for_fpr,
    reshape_df_to_get_1_cluster_per_row,
    get_primary_topic_mix_cols,
    create_dynamic_clusters_clean,
)

from subclu.models.reshape_clusters_v041 import (
    keep_only_target_labels,
    get_table_for_optimal_dynamic_cluster_params,
    get_dynamic_cluster_summary,
    flag_mature_clusters_to_exclude_from_qa,
)


setup_logging()
notebook_display_config()
print_lib_versions([gspread, pd, np])

python		v 3.7.13
===
gspread		v: 4.0.1
pandas		v: 1.3.5
numpy		v: 1.21.5


# Checklist to re-run for a country:

- change country name for google sheet name
- change country initial in google sheet
    - add google sheet KEY (after created)
- change country name in geo-relevance query

## Create google sheet for country outputs


In [8]:
%%time

country_name_sheet_ = 'Romania'
target_abbrev_ = 'RO'
GSHEET_KEY = '1smqquMCr5bV15S8eY12HG6ubAZ0kPPoAcCJDsE2F508'
GSHEET_NAME = f'i18n {country_name_sheet_} subreddits and clusters - model v0.4.1'


d_wsh_names = {
    'qa_ready': {
        'name': 'subs_need_to_be_rated',
    },
    'clusters_t2t_list_raw': {
        'name': f'raw_clusters_list_{target_abbrev_}_{target_abbrev_}',
    },
    'clusters_t2t_fpr_after_qa': {
        'name': f'fpr_clusters_after_qa_{target_abbrev_}_{target_abbrev_}',
    },
    'sub_raw': {
        'name': 'raw_data_per_subreddit',
    },
    # 'clusters_t2t_fpr_raw': {
    #     'name': f'raw_clusters_fpr_{target_abbrev_}_{target_abbrev_}',
    # },
}

if GSHEET_KEY is not None:
    sh = gc.open_by_key(GSHEET_KEY)
    print(f"Opening google worksheet: {GSHEET_NAME} ...")
else:
    print(f"** Creating google worksheet: {GSHEET_NAME} ...")
    sh = gc.create(GSHEET_NAME)

# create worksheets:
for _, d_ in d_wsh_names.items():
    sh_name = d_['name']
    try:
        d_['worksheet'] = sh.worksheet(sh_name)
        print(f"  Opening tab/sheet: {sh_name} ...")
    except Exception as e:
        print(f"  ** Creating tab/sheet: {sh_name} ...")
        d_['worksheet'] = sh.add_worksheet(sh_name, rows=5, cols=5)

if GSHEET_KEY is None:
    print(f"\n*** New sheet ID (assign it to GSHEET_KEY variable): ***\n{sh.id}\n")

Opening google worksheet: i18n Romania subreddits and clusters - model v0.4.1 ...
  Opening tab/sheet: subs_need_to_be_rated ...
  Opening tab/sheet: raw_clusters_list_RO_RO ...
  Opening tab/sheet: fpr_clusters_after_qa_RO_RO ...
  Opening tab/sheet: raw_data_per_subreddit ...
CPU times: user 65.6 ms, sys: 5.16 ms, total: 70.8 ms
Wall time: 1.06 s


# Load data from BigQuery

## Load subreddit geo-relevance & cultural relevance metadata

This data is already in bigQuery so read it straight from there. We'll use it to filter out geo-relevant (German) subs.

Also add the latest ratings so that we can filter based on those.

English-speaking countries don't have ambassador subs right now, so we should be able to create a standard template and replace the country name for these queries.

### SQL geo & cultural

In [9]:
%%time
%%bigquery df_geo_and_lang --project data-science-prod-218515 

-- Select geo+cultural subreddits for a target country
--  And add latest rating & over_18 flags to exclude X-rated & over_18
DECLARE TARGET_COUNTRY STRING DEFAULT 'Romania';
DECLARE MIN_COUNTRY_STANDARDIZED_RELEVANCE NUMERIC DEFAULT 3.0;


SELECT
    s.* EXCEPT(over_18, pt, verdict) 
    , nt.rating_name
    , nt.primary_topic
    , nt.rating_short
    , slo.over_18
    , slo.allow_discovery
    , CASE 
        WHEN(COALESCE(slo.over_18, 'f') = 't') THEN 'over_18_or_X_M_D_V'
        WHEN(COALESCE(nt.rating_short, '') IN ('X', 'M', 'D', 'V')) THEN 'over_18_or_X_M_D_V'
        ELSE 'unrated_or_E'
    END AS grouped_rating

FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` AS t
    -- Inner join b/c we only want to keep subs that are geo-relevant AND in topic model
    INNER JOIN (
        SELECT *
        FROM `reddit-employee-datasets.david_bermejo.subclu_subreddit_geo_score_standardized_20220212`
        WHERE country_name = TARGET_COUNTRY
    ) AS s
        ON t.subreddit_id = s.subreddit_id

    -- Add rating so we can get an estimate for how many we can actually use for recommendation
    LEFT JOIN (
        SELECT *
        FROM `data-prod-165221.ds_v2_postgres_tables.subreddit_lookup`
        -- Get latest partition
        WHERE dt = DATE(CURRENT_DATE() - 2)
    ) AS slo
    ON s.subreddit_id = slo.subreddit_id
    LEFT JOIN (
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE pt = DATE(CURRENT_DATE() - 2)
    ) AS nt
        ON s.subreddit_id = nt.subreddit_id

    -- Exclude popular US subreddits
    -- Can't query this table from local notebook because of errors getting google drive permissions. smh, excludefor now
    LEFT JOIN `reddit-employee-datasets.david_bermejo.subclu_subreddits_top_us_to_exclude_from_relevance` tus
        ON s.subreddit_name = LOWER(tus.subreddit_name)

WHERE 1=1
    AND s.subreddit_name != 'profile'
    AND COALESCE(s.type, '') = 'public'
    AND COALESCE(s.verdict, 'f') <> 'admin_removed'
    AND COALESCE(slo.over_18, 'f') = 'f'
    AND COALESCE(nt.rating_short, '') NOT IN ('X', 'D')

    AND(
        s.geo_relevance_default = TRUE
        OR s.relevance_percent_by_subreddit = TRUE
        OR s.e_users_percent_by_country_standardized >= MIN_COUNTRY_STANDARDIZED_RELEVANCE
    )
    AND country_name IN (
            TARGET_COUNTRY
        )

    AND (
         -- Exclude subs that are top in US but we want to exclude as culturally relevant
         --  For simplicity, let's go with the English exclusion (more relaxed) than the non-English one
         COALESCE(tus.english_exclude_from_relevance, '') <> 'exclude'
    )

ORDER BY e_users_percent_by_country_standardized DESC, users_l7 DESC, subreddit_name
;

CPU times: user 239 ms, sys: 29.6 ms, total: 268 ms
Wall time: 9.75 s


### Check df with geo + language information

In [10]:
print(df_geo_and_lang.shape)

(83, 26)


In [11]:
df_geo_and_lang.iloc[:4, :9]

,subreddit_id,subreddit_name,country_name,geo_relevance_default,b_users_percent_by_subreddit,e_users_percent_by_country_standardized,c_users_percent_by_country,d_users_percent_by_country_rank,relevance_percent_by_subreddit
0,t5_2qm35,romania,Romania,False,0.541680,10.569148,0.107762,1,True
1,t5_2bb1nb,romanianhotcelebs,Romania,False,0.643518,10.016344,0.015672,9,True
2,t5_jrd51,elrondnetwork,Romania,False,0.232289,8.818964,0.001082,684,True
3,t5_31h4ca,egld,Romania,False,0.247499,8.260017,0.000872,893,True


## Load model labels (clusters)

The clusters now live in a Big Query table and have standardized names, so pull the data from there.

### SQL labels


In [12]:
%%time
%%bigquery df_labels --project data-science-prod-218515 

-- select subreddit clusters from bigQuery

SELECT
    sc.subreddit_id
    , sc.subreddit_name
    , nt.primary_topic

    , sc.* EXCEPT(subreddit_id, subreddit_name, primary_topic_1214)
FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` sc
    LEFT JOIN (
        -- New view should be visible to all, but still comes from cnc_taxonomy_cassandra_sync
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE DATE(pt) = (CURRENT_DATE() - 2)
    ) AS nt
        ON sc.subreddit_id = nt.subreddit_id
;

CPU times: user 8.59 s, sys: 568 ms, total: 9.15 s
Wall time: 24.4 s


### Check label outputs

In [13]:
print(df_labels.shape)
df_labels.iloc[:4, :9]

(49558, 51)


,subreddit_id,subreddit_name,primary_topic,model_sort_order,posts_for_modeling_count,k_0013_label,k_0023_label,k_0041_label,k_0059_label
0,t5_5a9iie,progonlydj,None,40079,1000,12,19,34,49
1,t5_2x9c7,googleplaymusic,Music,40080,31,12,19,34,49
2,t5_3jzsk,ravedj,Music,40081,1000,12,19,34,49
3,t5_2rgie,happyhardcore,Music,40082,152,12,19,34,49


In [14]:
counts_describe(df_labels.iloc[:, :9])

,dtype,count,unique,unique-percent,null-count,null-percent
subreddit_id,object,"49,558","49,558",100.00%,0,0.00%
subreddit_name,object,"49,558","49,558",100.00%,0,0.00%
primary_topic,object,"41,366",52,0.13%,"8,192",16.53%
model_sort_order,int64,"49,558","49,558",100.00%,0,0.00%
posts_for_modeling_count,int64,"49,558",999,2.02%,0,0.00%
k_0013_label,int64,"49,558",13,0.03%,0,0.00%
k_0023_label,int64,"49,558",23,0.05%,0,0.00%
k_0041_label,int64,"49,558",41,0.08%,0,0.00%
k_0059_label,int64,"49,558",59,0.12%,0,0.00%


# Reshape data
Apply reshaping fxns so that we can export the data in a format that's good for QA.

## Keep only labels for Target subreddits


In [15]:
%%time
df_labels_target = keep_only_target_labels(
    df_labels=df_labels,
    df_geo=df_geo_and_lang,
    col_sort_order='model_sort_order',
    l_ix_subs=['subreddit_id', 'subreddit_name'],
    l_cols_to_front=None,
    geo_cols_to_drop=None,
)

0 <- subs to drop b/c they're not in model
(83, 74) <- df_labels_target.shape
CPU times: user 215 ms, sys: 3.8 ms, total: 219 ms
Wall time: 217 ms


In [16]:
counts_describe(df_labels_target.iloc[:, :15])

,dtype,count,unique,unique-percent,null-count,null-percent
model_sort_order,int64,83,83,100.00%,0,0.00%
subreddit_id,object,83,83,100.00%,0,0.00%
subreddit_name,object,83,83,100.00%,0,0.00%
primary_topic,object,65,24,36.92%,18,21.69%
rating_short,object,81,3,3.70%,2,2.41%
over_18,object,24,1,4.17%,59,71.08%
rating_name,object,81,3,3.70%,2,2.41%
posts_for_modeling_count,int64,83,69,83.13%,0,0.00%
k_0013_label,int64,83,11,13.25%,0,0.00%
k_0023_label,int64,83,16,19.28%,0,0.00%


## Run loop to find "optimal" min_num of subreddits for dynamic clusters


We want to balance two things:
- prevent orphan subreddits
- prevent clusters that are too large to be meaningful

In order to do this at a country level, we'll be better off starting with smallest cluster size and roll up until we have at least N subreddits in one cluster.

Find optimal `min_subreddits_in_cluster` based on:
- `orphan count`, 
- `number of clusters`,
- & other info

number might be different for each country and even within a country it might differ by when we filter NSFW subs.

In [17]:
col_new_cluster_val = 'cluster_label'
col_new_cluster_name = 'cluster_label_k'
col_new_cluster_prim_topic = 'cluster_majority_primary_topic'
col_new_cluster_topic_mix = 'cluster_topic_mix'

### Loop

In [18]:
%%time

df_optimal_min_check, n_min_subs_in_cluster_optimal = get_table_for_optimal_dynamic_cluster_params(
        df_labels_target=df_labels_target,
        col_new_cluster_val=col_new_cluster_val,
        col_new_cluster_name=col_new_cluster_name,
        col_new_cluster_prim_topic=col_new_cluster_prim_topic,
        col_new_cluster_topic_mix=col_new_cluster_topic_mix,
        min_subs_in_cluster_list=np.arange(3, 11),
        verbose=False,
        return_optimal_min_subs_in_cluster=True,
)

100%|██████████| 16/16 [00:00<00:00, 38.77it/s]

100%|██████████| 16/16 [00:00<00:00, 42.47it/s]

100%|██████████| 16/16 [00:00<00:00, 40.84it/s]

100%|██████████| 16/16 [00:00<00:00, 40.82it/s]

100%|██████████| 16/16 [00:00<00:00, 44.36it/s]

100%|██████████| 16/16 [00:00<00:00, 43.67it/s]

100%|██████████| 16/16 [00:00<00:00, 40.31it/s]

100%|██████████| 16/16 [00:00<00:00, 38.19it/s]

100%|██████████| 8/8 [00:06<00:00,  1.29it/s]


CPU times: user 6.07 s, sys: 150 ms, total: 6.22 s
Wall time: 6.34 s


In [19]:
assert (
    n_min_subs_in_cluster_optimal == df_optimal_min_check.loc[
        df_optimal_min_check['num_orphan_subreddits'] == df_optimal_min_check['num_orphan_subreddits'].min(),
        'min_subreddits_in_cluster'
    ].values[0]
)

In [20]:
df_optimal_min_check

,subs_to_cluster_count,min_subreddits_in_cluster,cluster_count,num_orphan_subreddits,num_subreddits_per_cluster_min,num_subreddits_per_cluster_mean,num_subreddits_per_cluster_median,num_subreddits_per_cluster_max,num_clusters_with_mature_primary_topic,cluster_ids_with_orphans
0,83,3,22,4,1,3.772727,4.0,13,3,"0001, 0002, 0005, 0009"
1,83,4,17,3,1,4.882353,5.0,13,4,"0001, 0002, 0005"
2,83,5,16,3,1,5.187500,5.5,13,4,"0001, 0002, 0005"
3,83,6,15,3,1,5.533333,6.0,13,3,"0001, 0002, 0005"
4,83,7,15,3,1,5.533333,6.0,13,3,"0001, 0002, 0005"
5,83,8,14,3,1,5.928571,5.5,13,3,"0001, 0002, 0005"
6,83,9,13,3,1,6.384615,5.0,19,3,"0001, 0002, 0005"
7,83,10,13,3,1,6.384615,5.0,19,3,"0001, 0002, 0005"


### Display loop results

In [21]:
def highlight_below_threshold(val, threshold=1):
    if val <= threshold:
        return "color:purple; font-weight: bold; background-color:yellow;"
    else:
        return ''
print(n_min_subs_in_cluster_optimal)

col_num_orph_subs = 'num_orphan_subreddits'
# col_num_subs_mean = 'num_subreddits_per_cluster_mean'
col_num_subs_median = 'num_subreddits_per_cluster_median'

style_df_numeric(
    df_optimal_min_check,
    rename_cols_for_display=True,
    l_bar_simple=[col_num_orph_subs,
                  col_num_subs_median,]
).applymap(highlight_below_threshold, subset=[col_num_orph_subs.replace('_', ' ')])


4


,subs to cluster count,min subreddits in cluster,cluster count,num orphan subreddits,num subreddits per cluster min,num subreddits per cluster mean,num subreddits per cluster median,num subreddits per cluster max,num clusters with mature primary topic,cluster ids with orphans
,,,,,,,,,,
0,83,3,22,4,1,3.77,4.00,13,3,"0001, 0002, 0005, 0009"
1,83,4,17,3,1,4.88,5.00,13,4,"0001, 0002, 0005"
2,83,5,16,3,1,5.19,5.50,13,4,"0001, 0002, 0005"
3,83,6,15,3,1,5.53,6.00,13,3,"0001, 0002, 0005"
4,83,7,15,3,1,5.53,6.00,13,3,"0001, 0002, 0005"
5,83,8,14,3,1,5.93,5.50,13,3,"0001, 0002, 0005"
6,83,9,13,3,1,6.38,5.00,19,3,"0001, 0002, 0005"
7,83,10,13,3,1,6.38,5.00,19,3,"0001, 0002, 0005"


## Get dyanimc clusters (apply optimal num from above)

side bar: about 57% of subreddits in Australia only had a single primary topic as their `topic_mix`, so combining `primary topic` might not give us as much info as we hoped.

At the same time, for 43% of subs we might get additional detail by combining the primary topics.


In [22]:
print(n_min_subs_in_cluster_optimal)

n_mix_start = 4
l_ix = ['subreddit_id', 'subreddit_name']
col_new_cluster_topic_mix = 'cluster_topic_mix'
col_subreddit_topic_mix = 'subreddit_full_topic_mix'
col_full_depth_mix_count = 'subreddit_full_topic_mix_count'
suffix_new_topic_mix = '_topic_mix_nested'
col_new_cluster_val_int = 'cluster_label_int'

df_labels_target_dynamic_raw = create_dynamic_clusters(
    df_labels_target,
    agg_strategy='aggregate_small_clusters',
    min_subreddits_in_cluster=n_min_subs_in_cluster_optimal,
    l_cols_labels_input=None,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    n_mix_start=n_mix_start,
    col_new_cluster_topic_mix=col_new_cluster_topic_mix,
    col_subreddit_topic_mix=col_subreddit_topic_mix,
    col_full_depth_mix_count=col_full_depth_mix_count,
    suffix_new_topic_mix=suffix_new_topic_mix,
    l_ix=l_ix,
    verbose=True,
)

01:28:02 | INFO | "Concat'ing nested cluster labels..."
01:28:02 | INFO | "Getting topic mix at different depths..."
01:28:02 | INFO | "  Assigning base topic mix cols"
01:28:02 | INFO | "  Creating deepest base topic mix col..."


4


01:28:03 | INFO | "  Iterating through additional subs with multiple topics..."
100%|██████████| 16/16 [00:00<00:00, 42.67it/s]
01:28:03 | INFO | "Initializing values for strategy: aggregate_small_clusters"
01:28:03 | INFO | "  Looping to roll-up clusters from smallest to largest..."
100%|██████████| 21/21 [00:00<00:00, 169.38it/s]
01:28:03 | INFO | "(83, 125) <- output shape"


In [23]:
style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_raw,
        return_dict=False,
    ),
    rename_cols_for_display=True,
)

,cluster count,num orphan subreddits,num subreddits per cluster min,num subreddits per cluster mean,num subreddits per cluster median,num subreddits per cluster max,num clusters with mature primary topic,cluster ids with orphans
,,,,,,,,
0,17,3,1,4.88,5.00,13,4,"0001, 0002, 0005"


In [91]:
# # # check column order
# style_df_numeric(
#     df_labels_target_dynamic_raw.iloc[104:109, :15],
#     rename_cols_for_display=True,
#     int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
#                     'by_country_rank',
#                     ],
#     pct_cols=['b_users_percent_by_subreddit',
#                   'c_users_percent_by_country',
#                   'users_percent_by_country_avg',
#                   ],
#     pct_labels='',
# )

### Minor QA checks

In [24]:
# # check column order
# style_df_numeric(
#     df_labels_target_dynamic_raw.iloc[70:74, -22:],
#     rename_cols_for_display=True,
#     int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
#                     'by_country_rank',
#                     ],
#     pct_cols=['b_users_percent_by_subreddit',
#                   'c_users_percent_by_country',
#                   'users_percent_by_country_avg',
#                   ],
#     pct_labels='',
# )

In [25]:
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix],
    top_n=9,
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
Gaming,16,19.3%,19.3%
Technology,14,16.9%,36.1%
"Place | Business, Economics, and Finance",13,15.7%,51.8%
Mature Themes and Adult Content,9,10.8%,62.7%
Music,6,7.2%,69.9%
Animals and Pets,6,7.2%,77.1%
Fashion,5,6.0%,83.1%
Crypto,5,6.0%,89.2%
Medical and Mental Health,4,4.8%,94.0%


In [26]:
# how many final clusters have multiple topics?
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix].str.count('\|')
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
0,70,84.3%,84.3%
1,13,15.7%,100.0%


In [27]:
# how many SUBREDDITS have multiple topics? (when we check the deepest clusters)
#  these two calls are equivalent

# value_counts_and_pcts(
#     df_labels_target_dynamic_raw[col_subreddit_topic_mix].str.count('\|')
# )

value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_full_depth_mix_count]
)

,subreddit_full_topic_mix_count-count,subreddit_full_topic_mix_count-percent,subreddit_full_topic_mix_count-pct_cumulative_sum
1,37,44.6%,44.6%
2,35,42.2%,86.7%
3,6,7.2%,94.0%
4,3,3.6%,97.6%
5,2,2.4%,100.0%


In [28]:
style_df_numeric(
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_full_depth_mix_count] >= 5]
    .iloc[-5:, :9]
    ,
    rename_cols_for_display=True,
)

,subreddit id,subreddit name,cluster label int,cluster topic mix,primary topic,rating short,subreddit full topic mix,rating name,over 18
,,,,,,,,,
27,t5_11mle5,thebackrooms,13,Mature Themes and Adult Content,-,E,Religion and Spirituality | Science | Gaming | History | Internet Culture and Memes,Everyone,-
41,t5_2qiwh,maps,10,Sports,Learning and Education,E,History | Place | Funny/Humor | Art | Learning and Education,Everyone,f


## Re-assign orphan subreddits (optional)

If there are subreddits that are orphan (see summary above), check them out to see if we can re-assign them w/o too much work. if we can't skip and move to the next country.

In [29]:
# # check subs around orphan sub
# # n_plus_minus_ = 5
# ix_orphan_ = (
#     df_labels_target_dynamic_raw
#     [df_labels_target_dynamic_raw[col_new_cluster_val] == '0011']
#     .index
# )
# df_labels_target_dynamic_raw.iloc[ix_orphan_, :9]

In [30]:
# # check other subs that are in the same cluster as orphan sub (at broadest level)
# l_cols_orphan_check = (
#     [
#         'subreddit_id',
#         col_new_cluster_topic_mix, 
#         # col_new_cluster_val,  # this can be really long and makes comparing harder
#         # col_subreddit_topic_mix,
#         'subreddit_name', 
#         col_new_cluster_name
#     ] +
#     l_cols_labels[:-5]
# )

# style_df_numeric(
#     df_labels_target_dynamic_raw
#     [df_labels_target_dynamic_raw['k_0013_label'] == 11]
#     [l_cols_orphan_check]
#     .iloc[3:14, :50]
#     ,
#     l_bar_simple=[c for c in l_cols_orphan_check[4:] if c.endswith('_label')],
#     rename_cols_for_display=True,

# )

In [31]:
# label_k_to_reassign_ = 'k_0320_label'
# label_val_to_reassign_ = '0011-0018-0032-0043-0046-0058-0062-0087-0244'
# subreddit_id_orphan_ = 't5_2tt7r'

# mask_orphan_and_new_group = (
#     (df_labels_target_dynamic_raw['subreddit_id'] == subreddit_id_orphan_) |
#     (
#         (df_labels_target_dynamic_raw[col_new_cluster_name] == label_k_to_reassign_) &
#         (df_labels_target_dynamic_raw[col_new_cluster_val] == label_val_to_reassign_)
#     )
# )

# # assign is similar to what we do in the dynamic function
# label_k_new_ = 'k_0118_label'
# label_val_new_col_ = f"{label_k_new_}_nested"
# new_prim_topic_col_ = label_k_new_.replace('_label', '_majority_primary_topic')
# c_update_topic_mix_ = label_k_new_.replace('_label', suffix_new_topic_mix)

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_name
# ] = label_k_new_

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_val
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][label_val_new_col_]

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_prim_topic
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][new_prim_topic_col_]

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_topic_mix
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][c_update_topic_mix_]

# del mask_orphan_and_new_group, label_k_to_reassign_, label_val_to_reassign_
# del label_k_new_, label_val_new_col_, new_prim_topic_col_

In [32]:
# # check again, num of orphans should be lower than before
# style_df_numeric(
#     get_dynamic_cluster_summary(
#         df_labels_target_dynamic_raw,
#         return_dict=False,
#     ),
#     rename_cols_for_display=True,
# )

In [33]:
# value_counts_and_pcts(
#     df_labels_target_dynamic_raw,
#     ['cluster_label'],
#     top_n=None,
#     return_df=True
# )['count'].describe()

## Get cluster for humans (list of subs in a cluster in a cell)
Here we get 1 cluster per row. 
Use cases:
- It makes it easier to quickly check NSFW clusters that we'll filter out
- we'll append the list of subreddit names from here to the final table for QA (makes it easier to evaluate whether the cluster makes sense).


In [36]:
col_subs_in_cluster_count = 'subs_in_cluster_count'
col_list_cluster_names = 'list_cluster_subreddit_names'

df_cluster_for_humans = reshape_df_to_get_1_cluster_per_row(
    df_labels_target_dynamic_raw,
    col_counterpart_count=col_subs_in_cluster_count,
    col_list_cluster_names=col_list_cluster_names,
    col_list_cluster_ids='list_cluster_subreddit_ids',
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_val_int=col_new_cluster_val_int,
    col_new_cluster_topic=col_new_cluster_topic_mix,
    verbose=False,
    get_one_column_per_sub_id=False,
)

(17, 7)


In [40]:
df_cluster_for_humans.iloc[10:15, :-1] # .iloc[40:45, :]

,cluster_label,cluster_label_k,cluster_topic_mix,cluster_label_int,subs_in_cluster_count,list_cluster_subreddit_names
10,0010,k_0013_label,Sports,10,2,"fauxcyrillic, maps"
11,0011-0018-0032-0043-0046,k_0063_label,Gaming,46,5,"forager, progressbar95, riskuniversalis, wyrmworks, critcrab"
12,0011-0018-0033,k_0041_label,Gaming,33,5,"discordadvertising, geforcenow, g2a, outlast, dukenukem"
13,0011-0018-0033-0048-0051-0064-0068-0096,k_0118_label,Gaming,96,6,"csgohacks, bulletforce, rustylake, deadbydaylightkillers, deadbydaylight, perkbydaylight"
14,0012,k_0013_label,Music,12,6,"rorep, yeat_, loatheband, hbo, asimov, gametoonsmemes"


In [41]:
# df_cluster_for_humans.tail(9)

### Check clusters that have mature topics

Make a list of cluster IDs to exclude for clean df

In [42]:
mask_mature_clusters_ = (
    df_cluster_for_humans[col_new_cluster_topic_mix].str.lower()
    .str.contains('mature')
)
print(f"{mask_mature_clusters_.sum()} <- clusters with 'Mature' in topic mix")

4 <- clusters with 'Mature' in topic mix


In [44]:
# check the first few clusters
(
    df_cluster_for_humans
    [mask_mature_clusters_]
    .iloc[:5, :-1]
)

,cluster_label,cluster_label_k,cluster_topic_mix,cluster_label_int,subs_in_cluster_count,list_cluster_subreddit_names
0,0001,k_0013_label,Mature Themes and Adult Content,1,1,romanianhotcelebs
1,0002,k_0013_label,Mature Themes and Adult Content,2,1,babyxzara
2,0005,k_0013_label,Mature Themes and Adult Content,5,1,goddess_network
8,0008-0013,k_0023_label,Mature Themes and Adult Content,13,6,"ghostposter, romanian, thebackrooms, neardeathexperiences, gifted, askoldpeopleadvice"


In [97]:
# (
#     df_cluster_for_humans
#     [mask_mature_clusters_]
#     .iloc[-12:, :]
# )

In [54]:
# might need to re-import in order for updates to show up
from subclu.models.reshape_clusters_v041 import _L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_, flag_mature_clusters_to_exclude_from_qa

print(len(_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_))
_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_[25:35]

71


['0001-0001-0003-0003-0003-0004-0005-0005',
 '0001-0001-0001-0001-0001-0002-0002-0002-0005-0009-0012-0014',
 '0001-0001-0002-0002-0002-0003-0003-0003-0009-0017',
 '0001-0001-0003-0003-0003-0004-0005-0005-0011-0020-0029',
 '0001-0001-0003-0003-0003-0004-0005-0005-0011-0020-0030',
 '0001-0001-0003-0003-0003-0004-0005-0005-0012',
 '0001-0001-0003-0003-0003-0004-0005-0005-0012-0022-0032',
 '0001-0001-0003-0003-0003-0004-0005-0005-0012-0023-0033',
 '0001-0001-0003-0003-0003-0004-0005-0005-0013',
 '0001-0001-0003-0003-0003-0004-0005-0005-0013-0026-0036-0043']

In [55]:
# sorted(_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_)

### Add the flag to exclude from QA & the list of sub names to df-raw

In [56]:
val_exclude_from_qa = 'exclude from QA'
col_exclude_from_qa = 'exclude_from_qa'


df_cluster_for_humans[col_exclude_from_qa] = flag_mature_clusters_to_exclude_from_qa(
    df_cluster_list=df_cluster_for_humans,
    col_new_cluster_val=col_new_cluster_val,
    col_exclude_from_qa=col_exclude_from_qa,
    val_exclude_from_qa=val_exclude_from_qa,
    list_known_nsfw_labels=_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_,
    depth_to_exclude=10,
    verbose=True,
)
print(f"{mask_mature_clusters_.sum()} <- clusters with 'Mature' in topic mix")
display(value_counts_and_pcts(df_cluster_for_humans[col_exclude_from_qa]))


df_labels_target_dynamic_raw = (
    df_labels_target_dynamic_raw
    # drop in case we're re-assigning the values
    .drop(
        [c for c in [col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa] if c in df_labels_target_dynamic_raw.columns],
        axis=1
    )
    .merge(
        df_cluster_for_humans[[col_new_cluster_val, col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa]],
        how='left',
        on=[col_new_cluster_val],
    )
)

df_labels_target_dynamic_raw.shape

4 <- clusters with 'Mature' in topic mix


,exclude_from_qa-count,exclude_from_qa-percent,exclude_from_qa-pct_cumulative_sum
keep,14,82.4%,82.4%
exclude from QA,3,17.6%,100.0%


(83, 128)

In [57]:
mask_mature_clusters_ = (
    df_cluster_for_humans[col_new_cluster_topic_mix].str.lower()
    .str.contains('mature')
)
print(f"{mask_mature_clusters_.sum()} <- clusters with 'Mature' in topic mix")

mask_keep_clusters = (
    df_cluster_for_humans[col_exclude_from_qa] != val_exclude_from_qa
)

print(f"\nCluster IDs that are mature but kept:")
# get list of cluster-IDs to add to exclude to mature list
(
    df_cluster_for_humans
    [mask_mature_clusters_ & mask_keep_clusters]
    ['cluster_label']
    .values
    [:15]
)

4 <- clusters with 'Mature' in topic mix

Cluster IDs that are mature but kept:


array(['0008-0013'], dtype=object)

In [58]:
(
    df_cluster_for_humans
    [mask_mature_clusters_ & mask_keep_clusters]
    .iloc[:5, :]
)

,cluster_label,cluster_label_k,cluster_topic_mix,cluster_label_int,subs_in_cluster_count,list_cluster_subreddit_names,list_cluster_subreddit_ids,exclude_from_qa
8,0008-0013,k_0023_label,Mature Themes and Adult Content,13,6,"ghostposter, romanian, thebackrooms, neardeathexperiences, gifted, askoldpeopleadvice","t5_39x2a, t5_2r3lm, t5_11mle5, t5_2u9p1, t5_2qzy8, t5_2jx9b6",keep


## Create new df_clean 

- Add list of subreddits to target-CLEAN, b/c we'll need it for rating final
- Add new columns & update order


### Copy baseline cols for clean

In [59]:
col_model_sort_order = 'model_sort_order'

df_labels_target_dynamic_raw, df_labels_target_dynamic_clean = create_dynamic_clusters_clean(
    df_dynamic_raw=df_labels_target_dynamic_raw,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    col_new_cluster_topic_mix=col_new_cluster_topic_mix,
    col_subreddit_topic_mix=col_subreddit_topic_mix,
)

# check num of orphans for clean
display(style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_clean.rename(columns={c: c.replace(' ', '_') for c in df_labels_target_dynamic_clean.columns}),
        return_dict=False,
    ),
    rename_cols_for_display=True,
).set_caption(f"Summary for df-CLEAN"))


# print(f"{len(l_cols_clean_final_for_qa)} <- expected final col count")
print(f"{df_labels_target_dynamic_raw.shape} <- df raw shape")
print(f"{df_labels_target_dynamic_clean.shape} <- df clean shape")

,cluster count,num orphan subreddits,num subreddits per cluster min,num subreddits per cluster mean,num subreddits per cluster median,num subreddits per cluster max,num clusters with mature primary topic,cluster ids with orphans
,,,,,,,,
0,14,0,2,5.71,5.00,13,1,


(83, 130) <- df raw shape
(80, 33) <- df clean shape


In [60]:
# [c for c in df_labels_target_dynamic_clean.columns if 'cluster' in c]

In [61]:
# df_labels_target_dynamic_clean.iloc[:5, :29]

In [62]:
# df_labels_target_dynamic_clean.iloc[:5, -10:]

## Create target to target list for FPR example

In [63]:
%%time
print(datetime.utcnow())

df_target_to_target_list = convert_distance_or_ab_to_list_for_fpr(
    df_labels_target_dynamic_clean.rename(columns={c: c.replace(' ', '_') for c in df_labels_target_dynamic_clean.columns}),
    convert_to_ab=True,
    col_counterpart_count='counterpart_count',
    col_list_cluster_names='list_cluster_subreddit_names',
    col_list_cluster_ids='list_cluster_subreddit_ids',
    l_cols_for_seeds=None,
    l_cols_for_clusters=None,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    verbose=False,
)
df_target_to_target_list.shape

2022-04-07 01:33:55.973235
  (468, 9) <- df_ab.shape after removing matches to self
  (63, 9) <- df_a_to_b.shape
CPU times: user 30.7 ms, sys: 0 ns, total: 30.7 ms
Wall time: 30.6 ms


In [64]:
df_target_to_target_list.drop('list_cluster_subreddit_ids', axis=1).iloc[:5, :11]

,subreddit_id_seed,subreddit_name_seed,cluster_label,cluster_label_k,primary_topic,cluster_majority_primary_topic,counterpart_count,list_cluster_subreddit_names
0,t5_2qhz5,asm,0006,k_0013_label,Programming,Technology,8,"needforspeed, skullcandy, ftc, asustuf, motorola, rpa, serialeturcestionline, dacia"
1,t5_2s94r,rpa,0006,k_0013_label,Technology,Technology,8,"needforspeed, skullcandy, ftc, asustuf, motorola, asm, serialeturcestionline, dacia"
2,t5_2qh70,yahoo,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology,Technology,4,"youtubemusic, bitdefender, invites, crt"
3,t5_35n4m,bitdefender,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology,Technology,4,"youtubemusic, invites, yahoo, crt"
4,t5_2sclw,crt,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology,Technology,4,"youtubemusic, bitdefender, invites, yahoo"


# Export data

Note that we have to use `fillna('')`

Otherwise, we'll get errors because the gspread library doesn't know how to handle `pd.NaN` or `np.Nan` (nulls).

## Define variables to create/access google sheet doc & worksheets

Moved the defintion to the top of the sheet so it's easier to automate.

## Save: Clean sheet to rate

In [65]:
d_wsh_names.keys()

dict_keys(['qa_ready', 'clusters_t2t_list_raw', 'clusters_t2t_fpr_after_qa', 'sub_raw'])

In [66]:
%%time
print(datetime.utcnow())
(
    d_wsh_names['qa_ready']['worksheet']
    .update([df_labels_target_dynamic_clean.columns.values.tolist()] + 
             df_labels_target_dynamic_clean.fillna('').values.tolist())
)

2022-04-07 01:35:11.225032
CPU times: user 28.9 ms, sys: 1.85 ms, total: 30.7 ms
Wall time: 365 ms


## Save: df cluster for humans

In [73]:
%%time
print(datetime.utcnow())
(
    d_wsh_names['clusters_t2t_list_raw']['worksheet']
    .update(
        [df_cluster_for_humans.rename(columns={c: c.replace('_', ' ') for c in df_cluster_for_humans}).columns.values.tolist()] + 
        df_cluster_for_humans.fillna('').values.tolist()
    )
)

2022-04-07 01:36:49.978273
CPU times: user 12 ms, sys: 84 µs, total: 12.1 ms
Wall time: 213 ms


## Save: target raw dynamic


In [68]:
l_cols_to_drop = (
    ['table_creation_date'] +
    [c for c in df_labels_target_dynamic_raw.columns if c.endswith('_nested')]
)
print(len(l_cols_to_drop))
# df_labels_target_dynamic_raw.columns.to_list()

45


In [69]:
%%time
print(datetime.utcnow())
(
    d_wsh_names['sub_raw']['worksheet']
    .update([df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).columns.values.tolist()] + 
             df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).fillna('').values.tolist())
)

2022-04-07 01:35:15.444400
CPU times: user 19.7 ms, sys: 1.97 ms, total: 21.7 ms
Wall time: 333 ms


## Save: FPR target-2-target as list

Even though data isn't fully ready, want to have the output ready to make sure it's in the right format that we need.

UPDATE: stop exporting this for now because it adds noise and could create confusion between it and the final QA sheet.

In [70]:
df_target_to_target_list.iloc[:5, :5]

,subreddit_id_seed,subreddit_name_seed,cluster_label,cluster_label_k,primary_topic
0,t5_2qhz5,asm,0006,k_0013_label,Programming
1,t5_2s94r,rpa,0006,k_0013_label,Technology
2,t5_2qh70,yahoo,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology
3,t5_35n4m,bitdefender,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology
4,t5_2sclw,crt,0006-0008-0013-0017-0017-0021-0022-0026,k_0118_label,Technology


In [71]:
# %%time
# print(datetime.utcnow())
# (
#     d_wsh_names['clusters_t2t_fpr_raw']['worksheet']
#     .update(
#         [df_target_to_target_list.columns.values.tolist()] + 
#         df_target_to_target_list.fillna('').values.tolist()
#     )
# )

### We can read the data back to confirm it's as expected

In [76]:
# Here's how to get the records as a dataframe
pd.DataFrame(
    d_wsh_names['qa_ready']['worksheet'].get_all_records()
).iloc[:5, :15]

,subreddit id,subreddit name,cluster label int,cluster topic mix,not country relevant,rated E,relevant to cluster/ other subreddits in cluster,safe to show in relation to cluster,country relevance notes,rating or cluster notes,link to sub,subs in cluster count,list cluster subreddit names,posts for modeling count,users l7
0,t5_2s8z6,needforspeed,6,Technology,,TRUE,,,,,www.reddit.com/r/needforspeed,9,"asm, rpa, motorola, asustuf, dacia, needforspeed, skullcandy, ftc, serialeturcestionline",719,59996
1,t5_2uelk,skullcandy,6,Technology,,TRUE,,,,,www.reddit.com/r/skullcandy,9,"asm, rpa, motorola, asustuf, dacia, needforspeed, skullcandy, ftc, serialeturcestionline",345,14287
2,t5_2sks6,ftc,6,Technology,,TRUE,,,,,www.reddit.com/r/ftc,9,"asm, rpa, motorola, asustuf, dacia, needforspeed, skullcandy, ftc, serialeturcestionline",493,7702
3,t5_2bblvy,asustuf,6,Technology,,TRUE,,,,,www.reddit.com/r/asustuf,9,"asm, rpa, motorola, asustuf, dacia, needforspeed, skullcandy, ftc, serialeturcestionline",249,7436
4,t5_2r9co,motorola,6,Technology,,TRUE,,,,,www.reddit.com/r/motorola,9,"asm, rpa, motorola, asustuf, dacia, needforspeed, skullcandy, ftc, serialeturcestionline",154,5484


# Appendix


## Additional checks on cluster depth

In [74]:
print(df_labels_target_dynamic_raw['cluster_label'].nunique())
display(
    value_counts_and_pcts(
        df_labels_target_dynamic_raw,
        ['cluster_label'],
        top_n=10,
    )
)
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    ['cluster_label'],
    top_n=None,
    return_df=True
)['count'].describe()

17


,count,percent,cumulative_percent
cluster_label,,,
0013-0023-0040-0058-0061-0077-0083-0114-0309-0633-0920-1026-1501-1775-2129-2269-2731-3031-3290-3572-3726-3788,13,15.7%,15.7%
0006,9,10.8%,26.5%
0011-0018-0033-0048-0051-0064-0068-0096,6,7.2%,33.7%
0013,6,7.2%,41.0%
0012,6,7.2%,48.2%
0008-0013,6,7.2%,55.4%
0009-0015-0028-0038-0040-0051-0054-0075-0211,5,6.0%,61.4%
0011-0018-0032-0043-0046,5,6.0%,67.5%
0011-0018-0033,5,6.0%,73.5%


count    17.000000
mean      4.882353
std       3.018326
min       1.000000
25%       3.000000
50%       5.000000
75%       6.000000
max      13.000000
Name: count, dtype: float64

### How deep are the clusters?



In [75]:
print(df_labels_target_dynamic_raw[col_new_cluster_name].nunique())
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    [col_new_cluster_name],
    top_n=None,
    sort_index=True,
)

7


,count,percent,cumulative_percent
cluster_label_k,,,
k_0013_label,33,39.8%,39.8%
k_0023_label,6,7.2%,47.0%
k_0041_label,10,12.0%,59.0%
k_0063_label,5,6.0%,65.1%
k_0118_label,11,13.3%,78.3%
k_0320_label,5,6.0%,84.3%
k_3927_label,13,15.7%,100.0%


In [ ]:
# style_df_numeric(
#     df_labels_target.tail(10),
#     # rename_cols_for_display=True,
#     l_bar_simple=[c for c in df_labels_target.columns if '_label' in c]
# )